# Common Random Variables
Polynomial chaos expansion (PCE) is a Hilbert space technique for random variables with finite variance.
Mathematically equivalent to Fourier series expansions for periodic signals, PCE allows to characterize a random variable in terms of its PCE coefficients (aka Fourier coefficients).
That is, the PCE of a random variable $\mathsf{x}$ is given by
$$
\mathsf{x} = \sum_{i=0}^L x_i \phi_i,
$$
where $x_i$ are the so-called PCE coefficients, and $\phi_i$ are the orthogonal polynomials that are orthogonal relative to the probability density function of $\mathsf{x}$.

This tutorial walks you through the PCE of common random variables, namely Gaussian (`gaussian`), Beta (`beta01`), Uniform(`uniform01`), Logistic (`logistic`), and shows how they are implemented in `PolyChaos`.

## Construction of Basis



In [1]:
using PolyChaos

The orthogonal polynomials are constructed using the `OrthoPoly`-type (here of degree at most `d`). For canonical measures special constructors are implemented:

In [2]:
d = 6

myops = Dict()
polynames = ["gaussian", "beta01", "uniform01", "logistic"]

# gaussian
gaussian = GaussOrthoPoly(d);
myops["gaussian"] = gaussian

# beta01
α, β = 1.3, 2.2
beta01 = Beta01OrthoPoly(d,α,β);
myops["beta01"] = beta01

# uniform01
uniform01 = Uniform01OrthoPoly(d);
myops["uniform01"] = uniform01

# logistic
logistic = LogisticOrthoPoly(d);
myops["logistic"] = logistic;

myops

Dict{Any,Any} with 4 entries:
  "uniform01" => Uniform01OrthoPoly{Array{Float64,1},Uniform01Measure,Quad{Floa…
  "logistic"  => LogisticOrthoPoly{Array{Float64,1},LogisticMeasure,Quad{Float6…
  "beta01"    => Beta01OrthoPoly{Array{Float64,1},Beta01Measure,Quad{Float64,Ar…
  "gaussian"  => GaussOrthoPoly{Array{Float64,1},GaussMeasure,Quad{Float64,Arra…

For example, let's evaluate the Gaussian basis polynomials at some points

In [3]:
points, degrees = randn(10), 0:2:d

[ evaluate(degree,points,gaussian) for degree in degrees ]

4-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]                                                                                                                                                            
 [1.202801415837944, -0.9980228340428031, 0.20922812599514273, -0.27303587033243937, 1.1735316146618584, -0.28179974420729514, -0.9996492608367771, 1.1076439449345306, 0.4509528657724353, 1.6254795664781185]
 [-5.364474417410014, 2.988140913442041, -2.7931360952731317, -0.8333079321820498, -5.3169500080365655, -0.7933899273355224, 2.997895688038623, -5.203700670987993, -3.6004529759413693, -5.859734444874583]   
 [22.029780924737587, -14.911086161975367, 19.249948619379953, 10.170416857963922, 22.213611760342687, 9.952293427751515, -14.98421858288123, 22.574030489217968, 21.768554830682078, 17.8472421561276]        

## Finding PCE Coefficients
Having constructed the orthogonal bases, the question remains how to find the PCE coefficients for the common random variables.
Every random variable can be characterized exactly by two PCE coefficients.
For a Gaussian random variable, this is familiar: the mean and the variance suffice to describe a Gaussian random variable entirely.
The same is true for any random variable of finite variance given the right basis.
The function `convert2affinePCE` provides the first two PCE coefficients (hence the name affine) for the common random variables.

### Gaussian
Given the Gaussian random variable $\mathsf{x} \sim \mathcal{N}(\mu, \sigma^2)$ with $\sigma > 0$, the affine PCE coefficients are

In [4]:
# Gaussian
μ, σ = 2., 0.2
pce_gaussian = convert2affinePCE(μ,σ,gaussian)

2-element Array{Float64,1}:
 2.0
 0.2

### Uniform
Given the uniform random variable $\mathsf{x} \sim \mathcal{U}(a, b)$ with finite support $a<b$, the affine PCE coefficients are

In [5]:
a, b = -0.3, 1.2
convert2affinePCE(a,b,uniform01)

2-element Array{Float64,1}:
 0.45
 1.5 

Instead, if the expected value and standard deviation are known, the affine PCE coefficients of the uniform random variable are

In [6]:
pce_uniform = convert2affinePCE(μ,σ,uniform01;kind="μσ")
# notice that the zero-order coefficient IS equivalent to the expected value μ

2-element Array{Float64,1}:
 2.0               
 0.6928203230275509

### Beta
Given the Beta random variable $\mathsf{x} \sim \mathcal{B}(a, b, \alpha, \beta)$ with finite support $a<b$ and shape parameters $\alpha, \beta > 0$, the affine PCE coefficients are

In [7]:
convert2affinePCE(a,b,beta01)

2-element Array{Float64,1}:
 0.2571428571428572
 1.5               

Instead, if the expected value and standard deviation are known, the affine PCE coefficients of the uniform random variable are

In [8]:
pce_beta = convert2affinePCE(μ,σ,beta01; kind="μσ")

2-element Array{Float64,1}:
 2.0               
 0.8780541105074454

### Logistic

Given the logstic random variable $\mathsf{x} \sim \mathcal{L}(a_1,a_2)$ where $a_2>0$ with the probability density function
$$
\rho(t) = \frac{1}{4 a_2} \, \operatorname{sech}^2 \left(\frac{t-a_1}{2a_2}\right)
$$
the affine PCE coefficients of the uniform random variable are

In [9]:
a1, a2 = μ, sqrt(3)*σ/pi
pce_logistic = convert2affinePCE(a1,a2,logistic)

2-element Array{Float64,1}:
 2.0                
 0.11026577908435842

## Moments
It is a key feature of PCE to compute moments from the PCE coefficients alone; no sampling is required.

### Gaussian

In [10]:
mean(pce_gaussian,myops["gaussian"]), std(pce_gaussian,myops["gaussian"])

(2.0, 0.2)

### Uniform

In [11]:
mean(pce_uniform,myops["uniform01"]), std(pce_uniform,myops["uniform01"])

(2.0, 0.19999999999999998)

### Beta

In [12]:
mean(pce_beta,myops["beta01"]), std(pce_beta,myops["beta01"])

(2.0, 0.20000000000000004)

### Logistic

In [13]:
mean(pce_logistic,myops["logistic"]), std(pce_logistic,myops["logistic"])

(2.0, 0.2)

## Sampling
Having found the PCE coefficients, it may be useful to sample the random variables.
That means, find $N$ realizations of the random variable that obey the random variable's probability density function.
This is done in two steps:
1. Draw $N$ samples from the measure (`sampleMeasure()`), and then
2. Evaluate the basis polynomials and multiply times the PCE coefficients, i.e. $\sum_{i=0}^L x_i \phi_i(\xi_j)$ where $\xi_j$ is the $j$-th sample from the measure (`evaluatePCE()`).

Both steps are combined in the function `samplePCE()`.

### Gaussian

In [14]:
using Statistics
N = 1000
ξ_gaussian = sampleMeasure(N,myops["gaussian"])
samples_gaussian = evaluatePCE(pce_gaussian,ξ_gaussian,myops["gaussian"])
# samplePCE(N,pce_gaussian,myops["gaussian"])

1000-element Array{Float64,1}:
 2.027829506511474 
 2.3505506958869455
 1.6519127674699183
 2.0306899952184883
 2.2623498572217833
 1.872040264273541 
 1.6221768005741044
 1.8082353411869314
 1.830474841177156 
 2.1975773986724914
 2.0884960032119486
 1.7777155851422968
 2.0718253727167295
 ⋮                 
 2.3438942755566488
 1.8898097049320415
 2.1957192187925485
 2.075597865375688 
 1.8129422869650513
 2.0751520918462587
 1.9738912926048482
 1.9282633697131966
 2.0797049768672204
 2.029868691610999 
 1.9074004016345567
 2.0416687821884083

### Uniform

In [15]:
ξ_uniform = sampleMeasure(N,myops["uniform01"])
samples_uniform = evaluatePCE(pce_uniform,ξ_uniform,myops["uniform01"])
# samples_uniform = samplePCE(N,pce_uniform,myops["uniform01"])

1000-element Array{Float64,1}:
 1.7271339218642832
 2.2265540408884528
 2.207598855982469 
 1.8450679500853044
 2.2137505990217536
 2.3246612766084733
 1.8172329050083509
 2.2193114177776545
 2.1092415058372507
 2.197667751189136 
 1.836635008448008 
 2.261092857529893 
 1.705952198365778 
 ⋮                 
 2.1189840132602487
 1.8244441070550297
 2.0995526908862994
 1.8061886348582161
 1.6820886779944582
 1.6547968643899527
 2.274867102113772 
 1.9668581036184074
 1.6669576209884998
 2.1352300930243624
 1.7532822852894292
 2.3315685436486073

### Beta

In [16]:
ξ_beta = sampleMeasure(N,myops["beta01"])
samples_beta = evaluatePCE(pce_beta,ξ_beta,myops["beta01"])
# samples_beta = samplePCE(N,pce_beta,myops["beta01"])

1000-element Array{Float64,1}:
 1.725996433223128 
 2.080671494974654 
 2.07443372323835  
 1.9179167905739314
 1.7259363587410204
 2.270123269678409 
 2.20256123196523  
 1.7071812384146907
 2.155640005220347 
 2.0060286830699026
 1.911357355191882 
 2.052871732171306 
 1.8999850540938654
 ⋮                 
 2.0372348843537047
 1.8459752527254811
 1.8859213864007358
 1.7646025557893341
 2.0291794317169445
 1.8948608581558348
 2.1259698212905174
 2.3208481628901674
 1.8339225271678894
 2.2559062715849474
 1.805267431794866 
 1.7553718790153154

### Logistic

In [17]:
ξ_logistic = sampleMeasure(N,myops["logistic"])
samples_logistic = evaluatePCE(pce_logistic,ξ_logistic,myops["logistic"])
# samples_logistic = samplePCE(N,pce_logistic,myops["logistic"])

1000-element Array{Float64,1}:
 2.059254264972423 
 1.8175292324256191
 2.1149584380750883
 2.309617625055588 
 1.8957399128754335
 2.1801211128448266
 1.7557604663921675
 2.185175951563327 
 1.9372226084167772
 2.389052188917093 
 2.0241765861725725
 2.2610914753795446
 1.8806412733548687
 ⋮                 
 1.7573730729254535
 2.106683787105582 
 2.029803509407892 
 2.0096958500503677
 2.0639266747463902
 1.9360368948800142
 1.9752671707271892
 1.8721070740738137
 1.8999603224017279
 2.2148943687482205
 2.2498365328226404
 1.8923748382046846